In [ ]:
from tensorflow.keras import optimizers

vgg_base_ft = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

for layer in vgg_base_ft.layers[-8:]:
    layer.trainable = True
for layer in vgg_base_ft.layers[:-8]:
    layer.trainable = False

model_ft = models.Sequential([
    vgg_base_ft,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_batches.num_classes, activation='softmax')
])

model_ft.compile(optimizer=optimizers.Adam(learning_rate=1e-5),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

print("Training Fine-Tuned Model (Unfrozen Last 2 Blocks)...")
history_ft = model_ft.fit(train_batches,
                          validation_data=valid_batches,
                          epochs=5,
                          verbose=1)

plt.figure(figsize=(8,6))
plt.plot(history_tl.history['val_accuracy'], label='Transfer Learning (Frozen Base)')
plt.plot(history_ft.history['val_accuracy'], label='Fine-Tuning (Unfrozen Last 2 Blocks)')
plt.title('Validation Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

test_loss, test_acc = model_ft.evaluate(test_batches)
print(f"\n\nTest Accuracy (Fine-Tuned Model): {test_acc:.4f}")

preds = model_ft.predict(test_batches)
pred_labels = np.argmax(preds, axis=1)
true_labels = test_batches.classes
class_names = list(test_batches.class_indices.keys())

plt.figure(figsize=(10,5))
for i in range(5):
    img_path = test_batches.filepaths[i]
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224,224))
    plt.subplot(1,5,i+1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"{class_names[pred_labels[i]]}\nT:{class_names[true_labels[i]]}")
plt.tight_layout()
plt.show()